In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict, ChainMap, Counter
import glob
import sys 
import functools
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import dask.dataframe as dd
from multiprocessing import Pool, Manager, cpu_count
from joblib import Parallel, delayed
import pymp
import gezi
from gezi import tqdm
tqdm.pandas()

In [2]:
timer = gezi.Timer('read user_action2.feather', True)
d = pd.read_feather('../input/user_action2.feather')
d.feedid = d.feedid.astype(int)
d.userid = d.userid.astype(int)
d.date_ = d.date_.astype(int)
timer.print()

read user_action2.feather start
read user_action2.feather duration: 20.262017965316772


In [5]:
d = d.sort_values(['date_'], ascending=True)

In [6]:
d.head()

,index,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite,version,finish_rate,stay_rate,is_first,actions
0,43864653,149954,47322,1,1,0,0,0,2000,2437,0,0,0,0,2,0.033333,0.040617,1,0
38,18085284,61932,105157,1,2,0,0,0,24474,24714,0,0,0,0,2,1.882615,1.901077,1,0
8,72819308,249037,29767,1,2,1,0,0,37368,38221,0,0,0,0,2,0.795064,0.813213,1,1
1,18085274,61932,40624,1,2,0,0,0,12690,12995,0,0,0,0,2,0.352500,0.360972,1,0
2,18085273,61932,49481,1,2,0,0,0,0,1172,0,0,0,0,2,0.000000,0.090154,1,0


In [7]:
len(set(d[d.date_ == 14].feedid) - set(d[d.date_ < 14].feedid))

3577

In [8]:
len(set(d[d.date_ == 14].feedid) - set(d[d.date_ < 14].feedid)) / len(set(d[d.date_ == 14].feedid))

0.05334029227557411

In [9]:
x = len((set(d[(d.date_ == 14) & (d.userid % 2 == 1)].feedid) - set(pd.concat([d[(d.date_ == 14) & (d.userid % 2 == 0)], d[d.date_ < 14]]).feedid)) )
x

152

In [10]:
x / len(set(d[(d.date_ == 14) & (d.userid % 2 == 1)].feedid))

0.002485812877164865

In [11]:
x2 = len((set(d[(d.date_ == 14) & (d.userid % 2 == 1)].feedid) - set(d[d.date_ < 14].feedid)))
x2

3425

In [12]:
len(set(d.feedid))

101820

In [13]:
num_feedids = len(open('../input/feed_embeddings.csv').readlines()) - 1
num_feedids

106444

In [14]:
len(set(d.feedid)) / num_feedids

0.9565593175754388

In [15]:
da = pd.read_csv('../input/test_a.csv')

In [16]:
len(set(da.feedid) - set(d.feedid)) / len(set(da.feedid))

0.03558868300876594

In [17]:
len(set(d.feedid) | set(da.feedid)) / num_feedids

0.9756679568599451

In [18]:
da1 = pd.read_csv('../input/v1/test_a.csv')

In [19]:
db1 = pd.read_csv('../input/v1/test_b.csv')

In [21]:
len(set(d.feedid) | set(da.feedid) | set(da1.feedid) | set(db1.feedid)) / num_feedids

0.975761902972455